In [1]:
import json
import os
import ast
from tqdm import tqdm
from multiprocessing import Pool
import sys
from random import choice, randint
import time
sys.path.append('/home/sreenath_a/Projects/OSS/Production/GitRipper')

import pandas as pd
from GitRipper.Ripper import collect

In [2]:
repos_data_path = '/home/sreenath_a/Projects/OSS/nsf-oss/Data_Collection/Python_data_collection/PyPi_metadata_flattened_wh.csv'
repo_owner_path = '/home/sreenath_a/Projects/OSS/Production/PyPi_Repo_Owner.csv'
repos_info_path = '/home/sreenath_a/Projects/OSS/Production/PyPi_GH_Data_Repos.csv'

In [3]:
# def get_owner_and_repo(url):
#     if type(url) != str:
#         return None, None
#     if "/" not in url:
#         return None, None
#     url = str(url).strip()
#     if url.endswith(".git"):
#         url = url[:-4]
#     owner = url.split("/")[-2]
#     repo = url.split("/")[-1]
#     return owner, repo

# def get_url(url_dict):
#     if url_dict is None:
#         return None
#     url_list = list(url_dict.values())
#     url_list = [i for i in url_list if "github.com" in i]
#     if len(url_list) == 0:
#         return None
#     u = [i for i in url_list if "github.com" in i.split("/")[-3]]
#     if len(u) == 0:
#         url = url_list[0]
#         url = "/".join(url.split("/")[:5])
#         if "github.com" in url.split("/")[2]:
#             return url
#         else:
#             return None
#     else:
#         url =  u[0]
#         return url

# # Create a blank csv file with the required columns: package_name, project_urls, owner, repo
# df = pd.DataFrame(columns=['package_name', 'project_urls', 'owner', 'repo'])
# df.to_csv(repo_owner_path, index=False)
# for df in tqdm(pd.read_csv(repos_data_path, chunksize=8000), total=450000/8000):
#     for row in df.iterrows():
#         try:
#             package_name = row[1]['PackageName']
#             project_urls = row[1]['project_urls']
#             if type(project_urls) != str:
#                 continue
#             owner, repo = get_owner_and_repo(get_url(ast.literal_eval(project_urls)))
#             if owner is None or repo is None:
#                 continue
#             df = pd.DataFrame([[package_name, project_urls, owner, repo]], columns=['package_name', 'project_urls', 'owner', 'repo'])
#             df.to_csv(repo_owner_path, mode='a', header=False, index=False)
#         except Exception as e:
#             print(e)
#             continue
# urls_json = choice(df.project_urls)
# if pd.isna(urls_json):
#     print("The url_dict is: None")
#     print("The url is: None")
# else:
#     urls_dict = ast.literal_eval(urls_json)
#     url = get_url(urls_dict)
#     print("The url_dict is: ", urls_dict)
#     print("The url is: ", url)

In [2]:
cpu_count = os.cpu_count()
print(cpu_count)

20


In [3]:
columns = ['owner', 'repo', 'name', 'description', 'licenseName', 'licenseSpdxId', 'licenseUrl',
           'shortDescriptionHTML', 'repourl', 'createdAt', 'updatedAt', 'pushedAt',
           'forkCount', 'stargazerCount', 'issuesCount', 'pullRequestsCount']

In [4]:
parallelcount = cpu_count
#parallelcount = 16

In [5]:
#Load the json file
with open('/home/sreenath_a/Projects/OSS/API_keys.json') as f:
    keys = json.load(f)['keys']

In [8]:
#create an empty dataframe and save as csv
if not os.path.exists(repos_info_path):
    df = pd.DataFrame(columns=columns)
    df.to_csv(repos_info_path, index=False)

In [9]:
#get all the packages alrady collected
slugs_collected = []
for df in pd.read_csv(repos_info_path, chunksize=1000):
    repo_names = df['repo'].tolist()
    repo_owners = df['owner'].tolist()
    for i in range(len(repo_names)):
        slugs_collected.append(str(repo_owners[i]).strip() + '/' + str(repo_names[i]).strip())
    slugs_collected = [i.strip() for i in list(set(slugs_collected))]

In [6]:
keys_list = [(i.split(":")[0], i.split(":")[1]) for i in keys]
collector = collect(keys_list)

Query failed with status code: 401
Key gkorkmaz failed
Query failed with status code: 401
Key Naskew failed
Query failed with status code: 401
Key rahuloms failed


In [11]:
def get_repo_info(owner, repo, key=None):
    time_to_sleep = randint(5, 10)
    try:
        owner = str(owner).strip()
        repo = str(repo).strip()
        if owner + '/' + repo in slugs_collected:
            return None
        time.sleep(time_to_sleep/10)
        repo_info = collector.getRepoInfo(owner, repo, key)
        del repo_info['readme']
    except Exception as e:
        print(e)
        return None
    return repo_info

In [12]:
def get_owner_and_repo(url):
    if type(url) != str:
        return None, None
    if "/" not in url:
        return None, None
    url = str(url).strip()
    if url.endswith(".git"):
        url = url[:-4]
    owner = url.split("/")[-2]
    repo = url.split("/")[-1]
    return owner, repo

In [13]:
total_pacakges = pd.read_csv(repo_owner_path).shape[0]
print(total_pacakges)

258573


In [14]:
print("Slugs collected: ", len(slugs_collected))
print("Packages to collect: ", total_pacakges - len(slugs_collected))

Slugs collected:  54539
Packages to collect:  204034


In [15]:
i = 0
best_keys = collector.getBestKeys(n=parallelcount)
for df in tqdm(pd.read_csv(repo_owner_path, chunksize=parallelcount), total=total_pacakges//parallelcount):
    df['slugs'] = df['owner'] + '/' + df['repo']
    df = df[~df['slugs'].isin(slugs_collected)]
    if df.shape[0] == 0:
        continue
    owners = df['owner'].tolist()
    repos = df['repo'].tolist()
    args = list(zip(owners, repos, best_keys))
    with Pool(parallelcount) as p:
        result = p.starmap(get_repo_info, args)
    result = [i for i in result if i is not None]
    if len(result) == 0:
        continue
    result_df = pd.DataFrame.from_dict(result, orient='columns')
    result_df = result_df[result_df.owner.notna()].reset_index(drop=True)
    if result_df.shape[0] == 0:
        continue
    result_df.to_csv(repos_info_path, mode='a', header=False, index=False)
    i = i+1
    if i%200 == 0:
        best_keys = collector.getBestKeys(n=parallelcount)
        waittime = randint(10, 60)
        time.sleep(waittime)

  0%|          | 0/12928 [00:00<?, ?it/s]

  0%|          | 32/12928 [00:33<4:01:36,  1.12s/it]

In [ ]:
df_repos = pd.read_csv(repos_info_path)
df_repos = df_repos[df_repos.owner.notna()].reset_index(drop=True)

In [ ]:
df_repos.shape

In [ ]:
df_repos.to_csv(repos_info_path, index=False)